## Importar librerías necesarias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Inspección y Limpieza del set de datos

In [ ]:
%run 2-Inspeccion_y_Limpieza.ipynb

## 3.EDA

### Valores nulos

Verificamos si existen valores nulos explícitos en cada variable.

In [ ]:
print("Valores nulos por columna:")
print(df_income.isnull().sum())

**Observación**

Aparentemente no hay valores nulos explícitos, pero da la posibilidad de tener valores nulos implícitos como:
- Valores fuera del rango lógico de cada variable _(Ej: Una edad negativa no es posible)_
- Discrepancias en mayúsculas/minúsculas _(Ej: Male o male)_
- Espacios en blanco o caracteres invisibles
- Codificacíón de las missing _(Ej: -99, missing, unknow, ...)_

En la exploración de los valores únicos presentes en cada categoría, vimos que algunos tienen una clase incognita "?" por lo que esto es una clara referencia a una codificación de valores nulos.
Por ello, vamos a darle un valor explícito y ver que podemos hacer con estos valores.

In [ ]:
# Reemplazar los valores '?' por NaN
df_income.replace('?', np.nan, inplace=True)

In [ ]:
# Contar los valores faltantes en cada columna
missing_values = df_income.isnull().sum()

# Filtrar las columnas que tienen valores faltantes
missing_columns = missing_values[missing_values > 0]

# Visualizar los valores faltantes
plt.figure(figsize=(8, 5))
ax = sns.barplot(x=missing_columns.index, y=missing_columns.values, palette="viridis")
plt.title('Número de Valores Faltantes por Columna')
plt.xlabel('Columnas')
plt.ylabel('Número de Valores Faltantes')
plt.xticks(rotation=45)

# Agregar anotaciones encima de cada barra
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'), 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'center', 
                xytext = (0, 9), 
                textcoords = 'offset points')

plt.show()

In [ ]:
import missingno

missingno.matrix(df_income.sort_values('Ocupación'))

Aparentemente hay una correlación entre Clase_Obrera y Ocupación, siendo esto una clara sugerencia de una dependencia lógica, es decir, cuando no se registra la clase obrera de una persona, tampoco se registra su ocupación.
- Este patrón podría indicar que estos campos se llenan juntos en el proceso de recolección de datos, o que conceptualmente están tan relacionados que cuando falta uno, naturalmente falta el otro.

Pero, si observamos la cantidad de valores faltantes de estas dos variables veremos que la Ocupación tiene 10 registros más de valores faltantes a comparación de la Clase_Obrera, pudiendo tal vez no ser del todo cierto la dependencia lógica. Sin embargo, vamos a asumir en primera instancia este comportamiento.

Además, la variable País_de_origen que también tiene valores faltantes aparantemente sus registros no siguen algún patron visual, pudiendo tratarse a estos valores nulos como un patron de tipo MCAR.

### Análisis univariado

In [ ]:
import plotly.express as px

# Definimos las columnas categóricas
categorical_columns = df_income.select_dtypes(include=['object']).columns

# Reemplazar los valores nulos en variables categóricas con 'NaN'
df_income[categorical_columns] = df_income[categorical_columns].fillna('NaN')

# Verificar valores únicos y su frecuencia para variables categóricas
for column in categorical_columns:
    # Graficar los valores únicos y su frecuencia
    value_counts = df_income[column].value_counts(dropna=False).reset_index()
    value_counts.columns = [column, 'Frecuencia']
    value_counts['Color'] = value_counts[column].apply(lambda x: 'red' if x == 'NaN' else 'blue')
    
    fig = px.bar(value_counts, x=column, y='Frecuencia', color='Color', 
                 title=f'Frecuencia de valores únicos en {column}', 
                 labels={column: column, 'Frecuencia': 'Frecuencia'})
    fig.update_layout(xaxis={'categoryorder':'total descending'})
    fig.show()

In [ ]:
# Visualización de distribuciones
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
sns.histplot(data=df_income, x='Edad', ax=axes[0,0], kde=True)
sns.histplot(data=df_income, x='Nivel_Educativo', ax=axes[0,1], kde=True)
sns.histplot(data=df_income, x='Ganancia_de_capital', ax=axes[1,0], kde=True)
sns.histplot(data=df_income, x='Pérdida_de_capital', ax=axes[1,1], kde=True)
sns.histplot(data=df_income, x='Horas_semanales', ax=axes[2,0], kde=True)
plt.tight_layout()
plt.show()

In [ ]:
# Seleccionar solo las columnas numéricas
numeric_columns = df_income.select_dtypes(include=['int64', 'float64']).columns

# Crear el gráfico de caja para cada columna numérica
df_income[numeric_columns].hist(figsize=(15, 10), bins=15, color='skyblue', edgecolor='black', linewidth=1.0)
plt.title('Gráfico de Caja de las Variables Numéricas')
plt.xticks(rotation=45)
plt.show()

### Análisis bivariado

In [ ]:
# Relación entre variables numéricas y el target (Ingresos)
fig, axes = plt.subplots(3, 2, figsize=(15, 10))
sns.boxplot(data=df_income, x='Ingresos', y='Edad', ax=axes[0,0])
sns.boxplot(data=df_income, x='Ingresos', y='Horas_semanales', ax=axes[0,1])
sns.boxplot(data=df_income, x='Ingresos', y='Ganancia_de_capital', ax=axes[1,0])
sns.boxplot(data=df_income, x='Ingresos', y='Pérdida_de_capital', ax=axes[1,1])
sns.boxplot(data=df_income, x='Ingresos', y='Nivel_Educativo', ax=axes[2,0])
plt.tight_layout()
plt.show()

# Calculamos la matriz de correlación
(
    df_income
    .replace(['<=50K', '>50K'], [0, 1])
    .corr(numeric_only=True)
    .pipe(
        lambda df: 
        sns.heatmap(
            df,
            annot=True,
            fmt=".2f",
            cmap='coolwarm',
            cbar=True,
            square=True
        )
    )
)

In [ ]:
for columna in categorical_columns.drop('Ingresos'):
    (
        pd.crosstab(
            df_income[columna], 
            df_income['Ingresos'], 
            normalize='index'
        )
        .sort_values(
            ['<=50K', '>50K'], 
            ascending=False
        )
        .plot(
            kind='barh', 
            figsize=(12, 10)
        )
    )
    plt.title(f'Tasa de Ingresos por {columna}')
    # plt.xticks(rotation=90, fontsize=16)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

### Eliminar los registros nulos (Opción 1 - NO óptima)

In [ ]:
print('Regitros antes de eliminar los valores faltantes:', df_income.shape[0])
df_income.replace('NaN', np.nan, inplace=True)
df_income.dropna(inplace=True)
print('Regitros después de eliminar los valores faltantes:', df_income.shape[0])

### Especificar una nueva categoría para los registros nulos (Opción 2 - NO óptima)

In [ ]:
# # Ponemos los valores nulos de forma explícita
# df_income.replace('NaN', np.nan, inplace=True)

# # Reemplazar los valores nulos en variables categóricas con 'Missing'
# df_income[categorical_columns] = df_income[categorical_columns].fillna('Missing')

# # Verificar la codificación
# df_income[categorical_columns].head()

### Imputar los registros nulos (Opción 3 - Óptima)